In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy as sp
import torch


In [60]:
raw_data=pd.read_csv('meta_data/Data1.csv')
label_1=pd.read_csv('meta_data/Label_of_Data1.csv')
label_1.iloc[86,2]=30014
length=label_1.iloc[:,2]-label_1.iloc[:,1]
length=pd.DataFrame(length,columns=['length']).to_numpy()
label_1=label_1.to_numpy()
label_1=np.append(label_1,length,axis=1)
label_1[:,0]=np.where(label_1[:,0]>'-1',1,0)
respond_label_1=label_1[:,0]
training_set_label=label_1[:,-1].reshape(126,1)

In [68]:
from sklearn.preprocessing import StandardScaler
#   进行标准化
data_set=raw_data.iloc[:,1:7].to_numpy()
response=raw_data.iloc[:,-2:]
Time=raw_data.iloc[:,0]
E_acc=(raw_data.iloc[:,1]**2 + raw_data.iloc[:,2]**2 + raw_data.iloc[:,3]**2).to_numpy().reshape(len(data_set),1)
E_omega=(raw_data.iloc[:,4]**2 + raw_data.iloc[:,5]**2 + raw_data.iloc[:,6]**2).to_numpy().reshape(len(data_set),1)
data_set=np.append(data_set,E_acc,axis=1)
data_set=np.append(data_set,E_omega,axis=1)
scaler=StandardScaler()

data_train1=scaler.fit_transform(data_set)  #   以每个点来做分类

training_set_label=scaler.fit_transform(training_set_label)
